In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Edge()
driver.get('https://printyourcopy.com/free-payslip-generator')
time.sleep(1)

#Clearing Function
def clear_values():
    
    driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[2]/textarea').send_keys(Keys.CONTROL + "a")  # Select all text
    driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[2]/textarea').send_keys(Keys.DELETE)        # Clear it
    driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[2]/textarea').send_keys(Keys.TAB)  
    
    for i in range(6):
        time.sleep(0.2)
        xpath = f'//*[@id="earnings.{i}.amount"]'
        driver.find_element(By.XPATH, xpath).send_keys(Keys.CONTROL + "a")  # Select all text
        driver.find_element(By.XPATH, xpath).send_keys(Keys.DELETE)        # Clear it
        driver.find_element(By.XPATH, xpath).send_keys(Keys.TAB)  

    for i in range(4):
        time.sleep(0.2)
        xpath = f'//*[@id="deductions.{i}.amount"]'
        driver.find_element(By.XPATH, xpath).send_keys(Keys.CONTROL + "a")  # Select all text
        driver.find_element(By.XPATH, xpath).send_keys(Keys.DELETE)        # Clear it
        driver.find_element(By.XPATH, xpath).send_keys(Keys.TAB)  

def data_precocessing(data):
    df = pd.read_excel(data)
    df['Date Of Joining'] = pd.to_datetime(df['Date Of Joining']).dt.strftime('%Y-%m-%d')
    columns_to_convert = ['Gross Salary', 'Bonus / Commission', 'Increment', 
                        'Reimbursment Amount', 'Compensation', 'Adjustments', 
                        'Absents Deduction', 'Lates Deduction']
    for col in columns_to_convert:
        df[col] = df[col].astype(int)

    return df
df = data_precocessing("Demo.xlsx")

def static_working():
    #Logo
    logo = os.path.abspath("Logo.jpg")
    driver.find_element(By.XPATH, '//*[@id="logo"]').send_keys(logo)

    #Employer Details
    employer_details = f"""
    Robust Support & Solutions
    Office No.404A, Fortune Tower
    PECHS Block 6, Karachi, Pakistan
    Phone & WhatsApp: 0311-3859635
    """
    driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[1]/textarea').send_keys(employer_details)
    
    xpaths = ['//*[@id="serial"]', '//*[@id="employeeSign"]', '//*[@id="employerSign"]', '//*[@id="currency"]']
    for xpath in xpaths:
        driver.find_element(By.XPATH, xpath).send_keys(Keys.CONTROL + "a")  # Select all text
        driver.find_element(By.XPATH, xpath).send_keys(Keys.DELETE)        # Clear it
        if xpath == '//*[@id="currency"]':
            driver.find_element(By.XPATH, xpath).send_keys("PKR")
        driver.find_element(By.XPATH, xpath).send_keys(Keys.TAB)   
        time.sleep(0.2)
    
    # Earning boxes
    n = 4
    for i in range(4):
        time.sleep(0.5)
        driver.find_element(By.XPATH, f'/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[2]/table/tbody/tr[{n}]/td/button').click()
        n+=1

    # Deduction boxes
    m = 11
    for i in range(3):
        time.sleep(0.5)
        driver.find_element(By.XPATH, f'/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[2]/table/tbody/tr[{m}]/td/button').click()
        m+=1

static_working()

time.sleep(0.5)

earnings_descriptions = [
    ("earnings.0.description", "Gross Salary", True),  # Requires clearing
    ("earnings.1.description", "Bonus / Commission", True),  # Requires clearing
    ("earnings.2.description", "Reimbursement", False),
    ("earnings.3.description", "Increment", False),
    ("earnings.4.description", "Compensation", False),
    ("earnings.5.description", "Adjustments", False)
]

deductions_descriptions = [
    ("deductions.0.description", "Absents", True),  # Requires clearing
    ("deductions.1.description", "Lates", False),
    ("deductions.2.description", "Payroll Tax", False),
    ("deductions.3.description", "Others", False)
]

# Function to update a description
def update_description(driver, field_xpath, description, clear_existing=False):
    """
    Updates the description field.

    :param driver: WebDriver instance
    :param field_xpath: The XPath of the field to update
    :param description: The description to input
    :param clear_existing: Whether to clear the field before entering text
    """
    element = driver.find_element(By.XPATH, f'//*[@id="{field_xpath}"]')
    if clear_existing:
        element.send_keys(Keys.CONTROL + "a")  # Select all text
        element.send_keys(Keys.DELETE)        # Clear it
    element.send_keys(description)
    element.send_keys(Keys.TAB)
    time.sleep(0.2)  # Sleep to allow UI updates

# Update earnings descriptions
for field_xpath, description, clear in earnings_descriptions:
    update_description(driver, field_xpath, description, clear_existing=clear)

# Update deductions descriptions
for field_xpath, description, clear in deductions_descriptions:
    update_description(driver, field_xpath, description, clear_existing=clear)


def input_values():
    for i in range(df.shape[0]):
        # Employee Details
        employee_details= f"""{df.loc[i, 'Full Name (As Per CNIC)']}
        {df.loc[i, 'CNIC Number']}
        {df.loc[i, 'Designation']}
        DOJ: {df.loc[i, 'Date Of Joining']}
        """
        driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[2]/textarea').send_keys(employee_details)

        driver.find_element(By.XPATH, '//*[@id="earnings.0.amount"]').send_keys(str(df.loc[i, 'Gross Salary']))
        driver.find_element(By.XPATH, '//*[@id="earnings.0.amount"]').send_keys(Keys.TAB)  
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="earnings.1.amount"]').send_keys(str(df.loc[i, 'Bonus / Commission']))
        driver.find_element(By.XPATH, '//*[@id="earnings.1.amount"]').send_keys(Keys.TAB)  
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="earnings.2.amount"]').send_keys(str(df.loc[i, 'Reimbursment Amount']))
        driver.find_element(By.XPATH, '//*[@id="earnings.2.amount"]').send_keys(Keys.TAB) 
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="earnings.3.amount"]').send_keys(str(df.loc[i, 'Increment']))
        driver.find_element(By.XPATH, '//*[@id="earnings.3.amount"]').send_keys(Keys.TAB) 
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="earnings.4.amount"]').send_keys(str(df.loc[i, 'Compensation']))
        driver.find_element(By.XPATH, '//*[@id="earnings.4.amount"]').send_keys(Keys.TAB) 
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="earnings.5.amount"]').send_keys(str(df.loc[i, 'Adjustments']))
        driver.find_element(By.XPATH, '//*[@id="earnings.5.amount"]').send_keys(Keys.TAB) 
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="deductions.0.amount"]').send_keys(str(df.loc[i, 'Absents Deduction']))
        driver.find_element(By.XPATH, '//*[@id="deductions.0.amount"]').send_keys(Keys.TAB)  
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="deductions.1.amount"]').send_keys(str(df.loc[i, 'Lates Deduction']))
        driver.find_element(By.XPATH, '//*[@id="deductions.1.amount"]').send_keys(Keys.TAB)  
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="deductions.2.amount"]').send_keys(str(df.loc[i, 'Payroll Tax Deduction']))
        driver.find_element(By.XPATH, '//*[@id="deductions.2.amount"]').send_keys(Keys.TAB) 
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="deductions.3.amount"]').send_keys(str(df.loc[i, 'Other Deductions']))
        driver.find_element(By.XPATH, '//*[@id="deductions.3.amount"]').send_keys(Keys.TAB) 
        time.sleep(0.3)

        # Check if the file exists and remove it if it does
        original_file = r"C:\Users\Desktop\Downloads\Payslip.pdf"
        if os.path.exists(original_file):
            os.remove(original_file)
            print(f"Removed previous Payslip file!")

        driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[2]/button[2]').click()
        time.sleep(2)
    
        new_file = fr'Payslips\{df.loc[i, "Full Name (As Per CNIC)"]} - {df.loc[i, "Month"]}.pdf'
        # Rename the file
        os.rename(original_file, new_file)
        print(f"File Directory change and Renamed: {original_file} -> {new_file}")

        clear_values()
        time.sleep(0.3)
input_values()

File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Muhammad Haris Dilshad - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Fahad Bin Zahid - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Ibrahim Najmuddin - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Zuhaib Ghori - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Muhammad Sabih Uddin Farooqi - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Muhammad Waqas Farooqi - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Rhonda Sacrafamilia - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Amir Bin Nasim - OCTOBER' 2024